In [169]:
import pdfplumber
import pandas as pd
from pdfreader import SimplePDFViewer
from bs4 import BeautifulSoup
import requests
from geopy.distance import geodesic
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm



warnings.filterwarnings('ignore')


budget_adopted_2023_2024_url = 'https://www.seattleschools.org/wp-content/uploads/2023/09/Budget-Book-Final_Web-Version-Adopted.pdf'
budget_summary_2023_2024_url = 'https://www.seattleschools.org/wp-content/uploads/2023/09/FY24-Adopted-Budget-Summary-Tables.pdf'
facilities_master_plan_2021_url = 'https://www.seattleschools.org/wp-content/uploads/2021/09/2021_Facilities_Master_Plan_Update.pdf'
attendance_area_report_url = 'https://www.seattleschools.org/wp-content/uploads/2022/08/Section-4-w-ADA-Cover.pdf'
budget='data/budget_adopted_2023_2024.pdf'
budget_summary='data/budget_summary_2023_2024.pdf'
facilities_master_plan = 'data/facilities_master_plan_2021.pdf'
attendance_area_report = 'data/attendance_area_report.pdf'

load_dotenv()



data_orig = pd.read_csv('data/sps_data_extract.csv')
data = data_orig.copy()

load_fact_schools = pd.read_csv('data/fact_schools.csv')
student_attending_live = pd.read_csv('data/fact_student_attending_live.csv')
student_live_attending = pd.read_csv('data/fact_student_live_attending.csv')

#Fact Schools - due to misspelled school names in the PDFs
fact_schools = pd.merge(load_fact_schools, load_fact_schools, how='left', left_on='Primary Index', right_on='Index')[['School_x','Index_x','School_y','Index_y', 'School Type_y']]\
    .rename(columns={'School_x':'Text','School_y':'School', 'Index_y':'Key', 'Index_x':'Index', 'School Type_y':'Type'})[['Key','Text','School','Index','Type']]

#Fact Students Living in Attendance Areas...
x = pd.merge(student_live_attending,fact_schools,right_on='Text', left_on='Attendance Area', how='left', suffixes=['_x','_y'])\
    .drop(columns=['Attendance Area','Text'])\
    .rename(columns={'School_y':'Attendance Area', 'School_x':'School'})[['Attendance Area','School','Count']]
y = pd.merge(x,fact_schools,right_on='Text', left_on='School', how='left', suffixes=['_x','_y'])\
    .drop(columns=['School_x','Text'])\
    .rename(columns={'School_y':'School'})[['Attendance Area','School','Count']]
fact_student_live_attending = y.copy()
fact_student_live_attending

#Fact Students Attending Schools Live... 
x = pd.merge(student_attending_live,fact_schools,right_on='Text', left_on='Attending School', how='left', suffixes=['_x','_y'])\
    .drop(columns=['Attending School','Text'])\
    .rename(columns={'School':'Attending School'})[['Attending School','Live Location','Count']]
y = pd.merge(x,fact_schools,right_on='Text', left_on='Live Location', how='left', suffixes=['_x','_y'])\
    .drop(columns=['Text', 'Live Location'])\
    .rename(columns={'School':'Live Location'})[['Attending School','Live Location','Count']]
fact_student_attending_live = y.copy()
fact_student_attending_live



########################################################################################
#   Functions listed here: 
########################################################################################

def get_lat_long(address, api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
    return None, None

def rename_school(s): 
    s1 = s.strip()
    return pd.merge(fact_schools[['School','Text']],pd.DataFrame(data={'School': [s1]}),left_on='Text',right_on='School',how='right')['School_x'][0]


def split_string(s):
    match = re.match(r"(.*?)(\d+)$", s)
    if match:
        non_numeric = match.group(1)
        numeric = match.group(2)
        return non_numeric, int(numeric)
    else:
        raise ValueError("The string does not end with a numeric part")
    

def create_df_from_split_string(s):
    df = pd.DataFrame([split_string(x) for x in s])
    df.columns = ['School','Count']
    return df.copy()




def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).miles


def convert_to_float(x):
    if isinstance(x, str):
        return float(x.replace(',', ''))
    return x




def extract_tables_from_pdf(pdf_path, page_num):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_num]
        tables = page.extract_tables()
        
        dataframes = []
        for table in tables:
            df = pd.DataFrame(table[1:], columns=table[0])
            dataframes.append(df)
        return dataframes

def find_dataframe_with_column(dfs, column_name):
    for idx, df in enumerate(dfs):
        if column_name in df.columns:
            return idx
    return -1  # Return -1 if the column name is not found in any DataFrame

def extract_text_from_pdf(pdf_path, page_num):
    with open(pdf_path, "rb") as file:
        viewer = SimplePDFViewer(file)
        viewer.navigate(page_num + 1)
        viewer.render()
        return "".join(viewer.canvas.strings)

def get_budget_table(page_num, include_school=True, pivoted=True):
    # First extract the tables from the PDF
    dataframes = extract_tables_from_pdf(budget, page_num)

    # Find the DataFrame that contains the budget table, it varies by page because of nuances with PDFPlumber
    budget_table_index = find_dataframe_with_column(dataframes,'School Year\nFunding Type 21-22 22-23 23-24')
    budget_table_b = dataframes[budget_table_index][[None]][None].str.split(' ', expand=True)
    budget_table_a = dataframes[budget_table_index].iloc[:, 0:1]

    # Overwrite the column names with the correct ones, Merge the two DataFrames    
    budget_table_a.columns = ['Budget']
    budget_table_b.columns = ['2021-22', '2022-23', '2023-24']
    budget_table = pd.merge(budget_table_a, budget_table_b, left_index=True, right_index=True)
    
    
    # Append the string BUDGET to indicate it is the budget data
    budget_table['Budget'] = budget_table['Budget'].apply(lambda x: x + ' (BUDGET)')

    # Clean up the DataFrame and cast numeric values to floats.
    budget_table['2021-22'] = budget_table['2021-22'].str.replace('$', '').str.replace(',', '').astype(float)
    budget_table['2022-23'] = budget_table['2022-23'].str.replace('$', '').str.replace(',', '').astype(float)
    budget_table['2023-24'] = budget_table['2023-24'].str.replace('$', '').str.replace(',', '').astype(float)


    if pivoted: 
        budget_table = budget_table.pivot_table(columns='Budget', values=['2023-24','2022-23','2021-22'], aggfunc='sum') 
        budget_table.columns.name = 'Index'
        budget_table = budget_table.reset_index().rename(columns={'index':'Year'})


    # Find the school name and add it to the DataFrame using a weird string in the PDF to identify the school name
    raw_text = extract_text_from_pdf(budget,page_num)
    if include_school:
        school_name = raw_text[0:raw_text.find('A.2023-24')]
        # Lots of akas for school names in the PDF and website
        if (school_name == 'Cascadia Elementrary'):
            school_name = 'Cascadia Elementary'
        elif (school_name == 'Rising Star Academy'): 
            school_name = 'Rising Star Elementary'
        elif (school_name == 'John Stanford Elementary'): 
            school_name = 'John Stanford International Elementary'
        elif (school_name == 'Northgate Elementary'): 
            school_name = 'James Baldwin Elementary'
        elif (school_name == 'McDonald Intl. Elementary'): 
            school_name = 'McDonald International Elementary'
        elif (school_name == 'Madrona K-5'): 
            school_name = 'Madrona Elementary'
        elif (school_name == 'Martin Luther King Jr. Elementary'): 
            school_name = 'Martin Luther King, Jr. Elementary'
        elif (school_name == 'Genesse Hill Elementary'): 
            school_name = 'Genesee Hill Elementary'
        elif (school_name == 'Dearborn Park Intl. Elementary'): 
            school_name = 'Dearborn Park International Elementary'
        elif (school_name == 'B.F. Day Elementary'): 
            school_name = 'Benjamin Franklin Day Elementary'
        elif (school_name == 'Concord Intl. Elementary'): 
            school_name = 'Concord International Elementary'
        elif (school_name == 'Franz Coe Elementary'): 
            school_name = 'Coe Elementary'
        elif (school_name == 'Beacon Hill Intl. Elementary'): 
            school_name = 'Beacon Hill International Elementary'
        budget_table['School'] = school_name
        col_to_move = budget_table.columns[-1]
        budget_table = budget_table[[col_to_move] + [col for col in budget_table.columns if col != col_to_move]]

    # Reorder the columns, Return the final dataframe
    return budget_table


def get_demographics_table(page_num, include_school=True, pivoted=True):
    # First extract the tables from the PDF
    dataframes = extract_tables_from_pdf(budget, page_num)

    # Find the DataFrame that contains the budget table, it varies by page because of nuances with PDFPlumber
    demographics_table_index = find_dataframe_with_column(dataframes,'School Year\n21-22 22-23 23-24')
    demographics_table_b = dataframes[demographics_table_index][[None]][None].str.split(' ', expand=True)
    demographics_table_a = dataframes[demographics_table_index].iloc[:, 0:1]    

    # Overwrite the column names with the correct ones, Merge the two DataFrames    
    demographics_table_a.columns = ['Demographic']
    demographics_table_b.columns = ['2021-22', '2022-23', '2023-24']
    demographics_table = pd.merge(demographics_table_a, demographics_table_b, left_index=True, right_index=True)

    # Append the string ENROLLMENT to indicate it is the enrollment data
    demographics_table['Demographic'] = demographics_table['Demographic'].apply(lambda x: x + ' (ENROLLMENT)')

    # Find the school name and add it to the DataFrame using a weird string in the PDF to identify the school name
    raw_text = extract_text_from_pdf(budget,page_num)

    if pivoted: 
        demographics_table = demographics_table.pivot_table(columns='Demographic', values=['2023-24','2022-23','2021-22'], aggfunc='sum') 
        demographics_table.columns.name = 'Index'
        demographics_table = demographics_table.reset_index().rename(columns={'index':'Year'})

    if include_school:
        school_name = raw_text[0:raw_text.find('A.2023-24')]
        # Cascadia Elementary is the only school with a typo in the PDF
        if (school_name == 'Cascadia Elementrary'):
            school_name = 'Cascadia Elementary'
        elif (school_name == 'Rising Star Academy'): 
            school_name = 'Rising Star Elementary'
        elif (school_name == 'John Stanford Elementary'): 
            school_name = 'John Stanford International Elementary'
        elif (school_name == 'Northgate Elementary'): 
            school_name = 'James Baldwin Elementary'
        elif (school_name == 'McDonald Intl. Elementary'): 
            school_name = 'McDonald International Elementary'
        elif (school_name == 'Madrona K-5'): 
            school_name = 'Madrona Elementary'
        elif (school_name == 'Martin Luther King Jr. Elementary'): 
            school_name = 'Martin Luther King, Jr. Elementary'
        elif (school_name == 'Genesse Hill Elementary'): 
            school_name = 'Genesee Hill Elementary'
        elif (school_name == 'Dearborn Park Intl. Elementary'): 
            school_name = 'Dearborn Park International Elementary'
        elif (school_name == 'B.F. Day Elementary'): 
            school_name = 'Benjamin Franklin Day Elementary'
        elif (school_name == 'Concord Intl. Elementary'): 
            school_name = 'Concord International Elementary'
        elif (school_name == 'Franz Coe Elementary'): 
            school_name = 'Coe Elementary'
        elif (school_name == 'Beacon Hill Intl. Elementary'): 
            school_name = 'Beacon Hill International Elementary'
        demographics_table['School'] = school_name
        col_to_move = demographics_table.columns[-1]
        demographics_table = demographics_table[[col_to_move] + [col for col in demographics_table.columns if col != col_to_move]]

    # Reorder the columns, Return the final dataframe
    return demographics_table




def get_facility_table(Use='E'):
    facility_inventory_table_page1 = 27
    facility_inventory_table_page2 = 28
    facility_inventory_table_page3 = 29

    a1 = extract_tables_from_pdf(facilities_master_plan,facility_inventory_table_page1)
    a2 = extract_tables_from_pdf(facilities_master_plan,facility_inventory_table_page2)
    a3 = extract_tables_from_pdf(facilities_master_plan,facility_inventory_table_page3)

    t1 = a1[0] # First and only table for each page is the one we want
    t2 = a2[0]
    t3 = a3[0]


    columns = ["Use","School",
            "Classification","Address",
            "Landmark","Building Area (Gross sf)",
            "Building Area (SCAP Recognized sf)",
            "Site Area (acre)","Date of Construction",
            "Date of Last Full Reno","Levy (1985-2019)"
            ]

    t1.columns = columns # Assigning column names to the table
    t2.columns = columns
    t3.columns = columns

    #t1.drop(0,inplace=True) 
    t2.drop(0,inplace=True) # Dropping the first row which contains the column names, but it gets the first table right for whatever reason.
    t3.drop(0,inplace=True)

    t3.drop(23,inplace=True) # Dropping the last row which contains the key/legend for the table

    t1.replace('','Y', regex=True, inplace=True) # Replacing the checkmark symbol with 'Y'
    t2.replace('','Y', regex=True, inplace=True)
    t3.replace('','Y', regex=True, inplace=True)

    t1['School'] = t1['School'].str.replace('\n',' ') # Removing the Newline character from the School column
    t1['School'] = t1['School'].str.replace('*','') # Removing the asterisk symbol from the School column
    t2['School'] = t2['School'].str.replace('\n',' ') # Removing the Newline character from the School column
    t2['School'] = t2['School'].str.replace('*','') # Removing the asterisk symbol from the School column
    t3['School'] = t3['School'].str.replace('\n',' ') # Removing the Newline character from the School column
    t3['School'] = t3['School'].str.replace('*','') # Removing the asterisk symbol from the School column

    t1_names_fixed = pd.merge(t1,fact_schools, left_on='School', right_on='Text',how='inner').query(f'Type == "{Use}"').drop(columns=['School_x','Index','Type','Text']).rename(columns={'School_y':'School'})
    t2_names_fixed = pd.merge(t2,fact_schools, left_on='School', right_on='Text',how='inner').query(f'Type == "{Use}"').drop(columns=['School_x','Index','Type','Text']).rename(columns={'School_y':'School'})
    t3_names_fixed = pd.merge(t3,fact_schools, left_on='School', right_on='Text',how='inner').query(f'Type == "{Use}"').drop(columns=['School_x','Index','Type','Text']).rename(columns={'School_y':'School'})
    t = pd.concat([t1_names_fixed,t2_names_fixed,t3_names_fixed])
    facility_inventory = pd.merge(fact_schools.query(f'Type == "{Use}" and Key == Index'),t, on='School', how='left').drop(columns=['Type','Index', 'Key_x', 'Key_y','Text'])
    facility_inventory['Levy (1985-2019)'] = facility_inventory['Levy (1985-2019)'].str.replace('\n',' ') # Removing the Newline character from the Levy column
    return facility_inventory



def redistribute_students(school_of_interest='Dunlap Elementary'):

        school_of_interest = rename_school(school_of_interest)
        try: 
                weighted_distribution_total = \
                        fact_student_attending_live.query(f'`Attending School` == "{school_of_interest}" and `Attending School` == `Live Location`')['Count'].values[0]
        except: 
                weighted_distribution_total = 1

        redistributed_students = \
                fact_student_attending_live.query(f'`Attending School` == "{school_of_interest}" and `Attending School` != `Live Location`')\
                .rename(columns={'Attending School':'From', 'Live Location':'To'})

        w1 = fact_student_live_attending.query(f'`Attendance Area` == "{school_of_interest}" and `Attendance Area` != `School`')
        w1['Weighted Count'] = (w1['Count'] / w1['Count'].sum()) * weighted_distribution_total
        w1 = w1.drop(columns=['Count']).rename(columns={'Attendance Area':'From', 'School':'To', 'Weighted Count':'Count'})[['From','To','Count']]

        redistributed_students = pd.concat([redistributed_students,w1], ignore_index=True).reset_index(drop=True)

        return redistributed_students.groupby('To')['Count'].sum().reset_index().sort_values(by='Count', ascending=False).reset_index(drop=True).copy()


def redistribute_students_vector(school_of_interest='Dunlap Elementary'):

    school_of_interest = rename_school(school_of_interest)
    try: 
            weighted_distribution_total = \
                    fact_student_attending_live.query(f'`Attending School` == "{school_of_interest}" and `Attending School` == `Live Location`')['Count'].values[0]
    except: 
            weighted_distribution_total = 1

    redistributed_students = \
            fact_student_attending_live.query(f'`Attending School` == "{school_of_interest}" and `Attending School` != `Live Location`')\
            .rename(columns={'Attending School':'From', 'Live Location':'To'})

    w1 = fact_student_live_attending.query(f'`Attendance Area` == "{school_of_interest}" and `Attendance Area` != `School`')
    w1['Weighted Count'] = (w1['Count'] / w1['Count'].sum()) * weighted_distribution_total
    w1 = w1.drop(columns=['Count']).rename(columns={'Attendance Area':'From', 'School':'To', 'Weighted Count':'Count'})[['From','To','Count']]

    redistributed_students = pd.concat([redistributed_students,w1], ignore_index=True).reset_index(drop=True)

    x = redistributed_students.groupby('To')['Count'].sum().reset_index().sort_values(by='Count', ascending=False).reset_index(drop=True)
    x['weighted'] = (x['Count'] / x['Count'].sum())
    x = x.drop(columns=['Count']).rename(columns={'To':'School','weighted':'Weight'})
    return pd.merge(x,pd.DataFrame(data['School'].unique(),columns=['School']),on='School',how='right').fillna(0).reset_index(drop=True).copy()


In [ ]:
########################################################################################
#   Build the Budget Table: 
########################################################################################
x = None

for i in range(67,129): #Pages 67-129 are for elementary schools
    if (x is None):
        x = get_budget_table(i)
    else:
        x = pd.concat([x,get_budget_table(i)], ignore_index=True)
    print(f"{i}, ", end="")
print("done")

In [ ]:
########################################################################################
#   Build the Demographics Table: 
########################################################################################
y = None

for i in range(67,129): #Pages 67-129 are for elementary schools
    if (y is None):
        y = get_demographics_table(i)
    else:
        y = pd.concat([y,get_demographics_table(i)], ignore_index=True)
    print(f"{i}, ", end="")
print("done")

In [ ]:
########################################################################################
#   Get Addresses and Lat/Longs 
########################################################################################
coords = None

for i in range(1,8): 

    # URL of the page to scrape    
    url = f'https://www.seattleschools.org/schools/type/elementary/page/{i}/'

    # Send a request to fetch the webpage content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request was unsuccessful

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements that contain school information
    school_elements = soup.find_all('div', class_='list-item')

    # Prepare lists to store school names and addresses
    schools = []
    addresses = []

    # Loop through each school element to extract names and addresses
    for school_element in school_elements:
        name_element = school_element.find('h4', class_='list-item-title').find('a')
        address_element = school_element.find('address')

        if name_element and address_element:
            school_name = name_element.get_text(strip=True)
            address = address_element.get_text(separator=" ", strip=True)
            
            schools.append(school_name)
            addresses.append(address)

    # Create a DataFrame from the extracted data
    data = {
        'School': schools,
        'Full Address': addresses
    }
    if coords is None:
        coords = pd.DataFrame(data)
    else: 
        coords = pd.concat([coords,pd.DataFrame(data)], ignore_index=True) 

# Remove trailing text from the addresses
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
coords['Full Address'] = coords['Full Address'].apply(lambda x: x[0:x.find('About')])
coords['Latitude'], coords['Longitude'] = zip(*coords['Full Address'].apply(lambda x: get_lat_long(x, api_key)))

In [ ]:
########################################################################################
#   Calculate nearest schools: 
########################################################################################


for index, row in coords.iterrows():
    min_distance = float('inf')
    closest_school = None
    school_coords = (row['Latitude'], row['Longitude'])

    for idx, other_row in coords.iterrows():
        if index != idx:
            other_school_coords = (other_row['Latitude'], other_row['Longitude'])
            distance = calculate_distance(school_coords, other_school_coords)
            if distance < min_distance:
                min_distance = distance
                closest_school = other_row['School']

    coords.at[index, 'Closest School'] = closest_school
    coords.at[index, 'Distance to Closest School (miles)'] = min_distance

In [ ]:
########################################################################################
#   Merge all DFs together, create one as 'z': 
########################################################################################


z = pd.merge(x,y, on=['School','Year'])
z = pd.merge(z,coords, how='left', on='School')

# Calculate helpful additional columns from existing data and cast numeric values to floats
z['Budget Efficiency'] = z['Total Budget (BUDGET)'].astype(float) / z['Total AAFTE* Enrollment (ENROLLMENT)'].astype(float)
z['Total AAFTE* Enrollment (ENROLLMENT)'] = z['Total AAFTE* Enrollment (ENROLLMENT)'].astype(float)

z.to_csv('data/sps_data_extract.csv', index=False)



In [ ]:
########################################################################################
#   Sandbox and Playground Below: 
########################################################################################

In [ ]:
z[z['School'] == 'Decatur Elementary']

# Group by school and calculate the average budget efficiency
school_performance = z.groupby('School').agg({
    'Budget Efficiency': 'mean',
    'Total AAFTE* Enrollment (ENROLLMENT)': 'mean',
    'Bilingual Education (BUDGET)': 'mean',
    'Special Education (BUDGET)': 'mean',
    'General Education (BUDGET)': 'mean'
}).reset_index()

# Sort schools by budget efficiency in descending order to identify top-performing schools
top_schools = school_performance.sort_values(by='Budget Efficiency', ascending=False).head(10)
top_schools

In [ ]:
# Calculate the change in enrollment for each school
enrollment_changes = z.pivot_table(index='School', columns='Year', values='Total AAFTE* Enrollment (ENROLLMENT)')

# Calculate the difference between the first and last year for each school
enrollment_changes['Change'] = enrollment_changes.iloc[:, -1] - enrollment_changes.iloc[:, 0]

# Sort schools by enrollment change in ascending order to find the sharpest decline
sharpest_decline = enrollment_changes.sort_values(by='Change').head(10)

# Sort schools by enrollment change in descending order to find the highest increase
highest_increase = enrollment_changes.sort_values(by='Change', ascending=False).head(10)

sharpest_decline, highest_increase

In [ ]:

# Study of Cascadia Elementary School - which was closed in 2023
cascadia_data = z[z['School'] == 'Cascadia Elementary']

# Extract relevant columns for Cascadia Elementary
cascadia_summary = cascadia_data[['Year', 'Total AAFTE* Enrollment (ENROLLMENT)', 'Budget Efficiency',
                                  'Bilingual Education (BUDGET)', 'Special Education (BUDGET)', 'General Education (BUDGET)']]

# Display Cascadia Elementary summary


In [ ]:

# Detailed trends for Cascadia Elementary

# Calculate year-on-year changes in enrollment and budget efficiency for Cascadia
cascadia_data['Enrollment Change'] = cascadia_data['Total AAFTE* Enrollment (ENROLLMENT)'].diff()
cascadia_data['Budget Efficiency Change'] = cascadia_data['Budget Efficiency'].diff()

# Calculate year-on-year changes in program funding
cascadia_data['Bilingual Education Change'] = cascadia_data['Bilingual Education (BUDGET)'].diff()
cascadia_data['Special Education Change'] = cascadia_data['Special Education (BUDGET)'].diff()
cascadia_data['General Education Change'] = cascadia_data['General Education (BUDGET)'].diff()

# Display detailed trends
cascadia_data[['Year', 'Total AAFTE* Enrollment (ENROLLMENT)', 'Enrollment Change', 'Budget Efficiency', 'Budget Efficiency Change',
               'Bilingual Education (BUDGET)', 'Bilingual Education Change', 'Special Education (BUDGET)', 'Special Education Change',
               'General Education (BUDGET)', 'General Education Change']]

In [ ]:
cascadia_data[['Year', 'Enrollment Change', 'Budget Efficiency Change',
                'Bilingual Education Change', 'Special Education Change',
               'General Education Change']]

In [153]:

########################################################################################
# Prepare data for Clustering Analysis
########################################################################################



data['Bilingual Education (BUDGET)'] = data['Bilingual Education (BUDGET)'].astype(float)
data['General Education (BUDGET)'] = data['General Education (BUDGET)'].astype(float)
data['Other Grants (BUDGET)'] = data['Other Grants (BUDGET)'].astype(float)
data['Special Education (BUDGET)'] = data['Special Education (BUDGET)'].astype(float)
data['State LAP (BUDGET)'] = data['State LAP (BUDGET)'].astype(float)
data['Total Budget (BUDGET)'] = data['Total Budget (BUDGET)'].astype(float)
data['Bilingual Education (ENROLLMENT)'] = data['Bilingual Education (ENROLLMENT)'].astype(float)
data['Free and Reduced Lunch (ENROLLMENT)'] = data['Free and Reduced Lunch (ENROLLMENT)'].astype(float)
data['Special Education (ENROLLMENT)'] = data['Special Education (ENROLLMENT)'].astype(float)
data['Total AAFTE* Enrollment (ENROLLMENT)'] = data['Total AAFTE* Enrollment (ENROLLMENT)'].astype(float)
data['Budget Efficiency'] = data['Budget Efficiency'].astype(float)
data['Distance to Closest School (miles)'] = data['Distance to Closest School (miles)'].astype(float)



# Normalize the numeric data using MinMaxScaler in order to apply K-means clustering fairly on all features
#scaler = MinMaxScaler()
#numeric_df = data.select_dtypes(include=['float64'])
#df_normalized = pd.DataFrame(scaler.fit_transform(numeric_df), columns=numeric_df.columns)
#data = pd.merge(df_normalized, data.drop(columns=numeric_df.columns), left_index=True, right_index=True)



In [ ]:
########################################################################################
# Clustering Analysis A: uses all budget and enrollment columns, and budget efficiency
########################################################################################


# Select relevant columns for clustering
columns_to_cluster = [
    'Bilingual Education (BUDGET)', 'General Education (BUDGET)', 'Other Grants (BUDGET)', 
    'Special Education (BUDGET)', 'State LAP (BUDGET)', 'Total Budget (BUDGET)', 
    'Bilingual Education (ENROLLMENT)', 'Free and Reduced Lunch (ENROLLMENT)', 
    'Special Education (ENROLLMENT)', 'Total AAFTE* Enrollment (ENROLLMENT)', 
    'Budget Efficiency'
]

# Handle missing values by filling with the mean of each column
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data[columns_to_cluster])

# Normalize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    sse.append(kmeans.inertia_)


plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.title('Analysis A: Elbow Method to Determine Optimal k for Clustering')
plt.xlabel('Number of Clusters')
plt.ylabel('Sum of Squared Distances')
plt.show()


# Apply K-means clustering with k=3
kmeans_3 = KMeans(n_clusters=3, random_state=42)
clusters_3 = kmeans_3.fit_predict(data_scaled)

# Apply K-means clustering with k=4
kmeans_4 = KMeans(n_clusters=4, random_state=42)
clusters_4 = kmeans_4.fit_predict(data_scaled)

# Apply K-means clustering with k=5
kmeans_5 = KMeans(n_clusters=5, random_state=42)
clusters_5 = kmeans_5.fit_predict(data_scaled)
                                  
# Apply K-means clustering with k=6
kmeans_6 = KMeans(n_clusters=6, random_state=42)
clusters_6 = kmeans_6.fit_predict(data_scaled)


# Add the cluster labels to the original dataframe
data['Cluster_3a'] = clusters_3
data['Cluster_4a'] = clusters_4
data['Cluster_5a'] = clusters_5
data['Cluster_6a'] = clusters_6





In [ ]:

# Analyze the clusters by looking at the mean values of the features for each cluster
cluster_3a_summary = data.groupby('Cluster_3a')[columns_to_cluster].mean()
cluster_4a_summary = data.groupby('Cluster_4a')[columns_to_cluster].mean()
cluster_5a_summary = data.groupby('Cluster_5a')[columns_to_cluster].mean()
cluster_6a_summary = data.groupby('Cluster_6a')[columns_to_cluster].mean()

#cluster_3a_summary, cluster_4a_summary, cluster_5a_summary, cluster_6a_summary


In [ ]:
########################################################################################
## Observations from Clustering Analysis A: 
########################################################################################


#3 Clusters:
#Cluster 0: Lower budgets and enrollments.
#Cluster 1: Highest budgets and enrollments, especially in bilingual and special education.
#Cluster 2: Moderate budget, high total enrollment.


#4 Clusters:
#Cluster 0: High bilingual education budgets.
#Cluster 1: High budgets and enrollments, significant special education funding.
#Cluster 2: Lower budgets and enrollments.
#Cluster 3: Moderate budgets and enrollments.

#5 Clusters:
#Cluster 0: High overall budgets and enrollments, particularly in general education and other grants.
#Cluster 1: Moderate budgets and enrollments, with a significant amount of state LAP funding.
#Cluster 2: Highest budgets, particularly in special education and state LAP, with high enrollments.
#Cluster 3: High special education funding and enrollment, moderate budgets.
#Cluster 4: Lower budgets and enrollments, with moderate funding in special education and state LAP.

#6 Clusters:
#Cluster 0: High overall budgets and enrollments, with a significant budget for special education and state LAP.
#Cluster 1: Moderate budgets and enrollments, with a notable budget for other grants and a lower budget efficiency.
#Cluster 2: Moderate budgets and enrollments with a focus on bilingual education and state LAP.
#Cluster 3: Higher budgets and enrollments, particularly in special education and state LAP.
#Cluster 4: Highest budgets and enrollments, especially in bilingual education and state LAP.
#Cluster 5: Moderate budgets and enrollments, with a significant budget for other grants and a lower budget efficiency.


In [ ]:
########################################################################################
### Clustering Analysis B: excludes state LAP funding, and includes distance to closest school
########################################################################################

columns_to_cluster = [
    'General Education (BUDGET)', 'Other Grants (BUDGET)', 'Special Education (BUDGET)',
    'Total Budget (BUDGET)', 'Federal Title I (BUDGET)', 'Bilingual Education (ENROLLMENT)',
    'Free and Reduced Lunch (ENROLLMENT)', 'Special Education (ENROLLMENT)',
    'Total AAFTE* Enrollment (ENROLLMENT)', 'Budget Efficiency', 'Distance to Closest School (miles)'
]


# Handle missing values by filling with the mean of each column
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data[columns_to_cluster])

# Normalize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    sse.append(kmeans.inertia_)


plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.title('Analysis B: Elbow Method to Determine Optimal k for Clustering')
plt.xlabel('Number of Clusters')
plt.ylabel('Sum of Squared Distances')
plt.show()




# Apply K-means clustering with k=3
kmeans_3 = KMeans(n_clusters=3, random_state=42)
clusters_3 = kmeans_3.fit_predict(data_scaled)

# Apply K-means clustering with k=4
kmeans_4 = KMeans(n_clusters=4, random_state=42)
clusters_4 = kmeans_4.fit_predict(data_scaled)

# Apply K-means clustering with k=5
kmeans_5 = KMeans(n_clusters=5, random_state=42)
clusters_5 = kmeans_5.fit_predict(data_scaled)
                                  
# Apply K-means clustering with k=6
kmeans_6 = KMeans(n_clusters=6, random_state=42)
clusters_6 = kmeans_6.fit_predict(data_scaled)


# Add the cluster labels to the original dataframe
data['Cluster_3b'] = clusters_3
data['Cluster_4b'] = clusters_4
data['Cluster_5b'] = clusters_5
data['Cluster_6b'] = clusters_6

# Analyze the clusters by looking at the mean values of the features for each cluster
cluster_3b_summary = data.groupby('Cluster_3b')[columns_to_cluster].mean()
cluster_4b_summary = data.groupby('Cluster_4b')[columns_to_cluster].mean()
cluster_5b_summary = data.groupby('Cluster_5b')[columns_to_cluster].mean()
cluster_6b_summary = data.groupby('Cluster_6b')[columns_to_cluster].mean()

#cluster_3b_summary, cluster_4b_summary, cluster_5b_summary, cluster_6b_summary

In [ ]:
########################################################################################
## Observations from Clustering Analysis B:
########################################################################################


#3 Clusters:
#Cluster 0: High budgets in General Education and Other Grants, moderate Special Education budget, relatively high Total Budget. Moderate budget efficiency and average distance to the closest school.
#Cluster 1: Moderate budgets across all categories, lower Total Budget. Moderate budget efficiency and slightly higher distance to the closest school.
#Cluster 2: High budgets in General Education, low in Other Grants, high Special Education budget, highest Total Budget. Lowest budget efficiency and shortest distance to the closest school.
# # # Cluster 2 most likely to be targets for closure. 
# # # Cluster 0 most likely not to be closed.

#4 Clusters:
#Cluster 0: High budgets in General Education and Other Grants, moderate Special Education budget, high Total Budget. Moderate budget efficiency and average distance to the closest school.
#Cluster 1: Moderate budgets in General Education, low in Other Grants, moderate Special Education budget, moderate Total Budget. Low budget efficiency and longer distance to the closest school.
#Cluster 2: High budgets in General Education, low in Other Grants, high Special Education budget, highest Total Budget. Low budget efficiency and average distance to the closest school.
#Cluster 3: Low budgets across all categories, lowest Total Budget. Moderate budget efficiency and shortest distance to the closest school.
# # # Clusters 2 and 3 most likely to be targets for closure.
# # # Clusters 0 and 1 likely not to be closed.

#5 Clusters:
#Cluster 0: High budgets in General Education and Other Grants, low Special Education budget, moderate Total Budget. High budget efficiency and moderate distance to the closest school.
#Cluster 1: Moderate budgets in General Education, low in Other Grants, moderate Special Education budget, moderate Total Budget. Low budget efficiency and longer distance to the closest school.
#Cluster 2: High budgets in General Education, moderate in Other Grants, high Special Education budget, high Total Budget. Moderate budget efficiency and average distance to the closest school.
#Cluster 3: Low budgets across all categories, low Total Budget. Moderate budget efficiency and shortest distance to the closest school.
#Cluster 4: High budgets in General Education, low in Other Grants, high Special Education budget, highest Total Budget. Lowest budget efficiency and average distance to the closest school.
# # # Clusters 3 and 4 most likely to be targets for closure.
# # # Clusters 0 and 2 likely not to be closed. 

#6 Clusters:
#Cluster 0: High budgets in General Education and Other Grants, low Special Education budget, moderate Total Budget. High budget efficiency and longer distance to the closest school.
#Cluster 1: Moderate budgets in General Education, low in Other Grants, moderate Special Education budget, moderate Total Budget. Low budget efficiency and longer distance to the closest school.
#Cluster 2: Highest General Education budget, moderate in Other Grants, highest Special Education budget, highest Total Budget. Low budget efficiency and longer distance to the closest school.
#Cluster 3: Low budgets across all categories, lowest Total Budget. Moderate budget efficiency and shortest distance to the closest school. Low budget efficiency and longest distance to the closest school.
#Cluster 4: High budgets in General Education, low in Other Grants, high Special Education budget, high Total Budget. Lowest budget efficiency and average distance to the closest school.
#Cluster 5: High budgets in General Education, low in Other Grants, high Special Education budget, high Total Budget. Moderate budget efficiency and shortest distance to the closest school.
# # # Cluster 4 and 5 most likely to be targets for closure.
# # # Clusters 0 and 2 most likely not to be closed. 


In [ ]:
########################################################################################
### Plots for B analysis
########################################################################################



scatter_6 = data[['School','Year','Cluster_6b', 'Distance to Closest School (miles)', 'Budget Efficiency']]
scatter_5 = data[['School','Year','Cluster_5b', 'Distance to Closest School (miles)', 'Budget Efficiency']]
scatter_4 = data[['School','Year','Cluster_4b', 'Distance to Closest School (miles)', 'Budget Efficiency']]
scatter_3 = data[['School','Year','Cluster_3b', 'Distance to Closest School (miles)', 'Budget Efficiency']]

scatter_2023_6b = scatter_6[scatter_6['Year']=='2023-24'].sort_values(by='Cluster_6b')
scatter_2022_6b = scatter_6[scatter_6['Year']=='2022-23'].sort_values(by='Cluster_6b')
scatter_2021_6b = scatter_6[scatter_6['Year']=='2021-22'].sort_values(by='Cluster_6b')

scatter_2023_5b = scatter_5[scatter_5['Year']=='2023-24'].sort_values(by='Cluster_5b')
scatter_2022_5b = scatter_5[scatter_5['Year']=='2022-23'].sort_values(by='Cluster_5b')
scatter_2021_5b = scatter_5[scatter_5['Year']=='2021-22'].sort_values(by='Cluster_5b')

scatter_2023_4b = scatter_4[scatter_4['Year']=='2023-24'].sort_values(by='Cluster_4b')
scatter_2022_4b = scatter_4[scatter_4['Year']=='2022-23'].sort_values(by='Cluster_4b')
scatter_2021_4b = scatter_4[scatter_4['Year']=='2021-22'].sort_values(by='Cluster_4b')

scatter_2023_3b = scatter_3[scatter_3['Year']=='2023-24'].sort_values(by='Cluster_3b')
scatter_2022_3b = scatter_3[scatter_3['Year']=='2022-23'].sort_values(by='Cluster_3b')
scatter_2021_3b = scatter_3[scatter_3['Year']=='2021-22'].sort_values(by='Cluster_3b')




In [ ]:
########################################################################################
### K=6 Clusters B Analysis Plot
########################################################################################


df = pd.DataFrame(scatter_2023_6b)

# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale

cluster_0_size = scatter_2023_6b[scatter_2023_6b['Cluster_6b']==0].shape[0]
cluster_1_size = scatter_2023_6b[scatter_2023_6b['Cluster_6b']==1].shape[0]
cluster_2_size = scatter_2023_6b[scatter_2023_6b['Cluster_6b']==2].shape[0]
cluster_3_size = scatter_2023_6b[scatter_2023_6b['Cluster_6b']==3].shape[0]
cluster_4_size = scatter_2023_6b[scatter_2023_6b['Cluster_6b']==4].shape[0]
cluster_5_size = scatter_2023_6b[scatter_2023_6b['Cluster_6b']==5].shape[0]


labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})', 
            f'Cluster 3 ({cluster_3_size})',
            f'Cluster 4 ({cluster_4_size})',
            f'Cluster 5 ({cluster_5_size})'
          ]
colors = ['green', 'gray', 'green', 'gray', 'red', 'red']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
#markers = ['o', 's', '^', 'P', 'D', 'X']
markers = ['P', 'P', 'P', 'P', 'o', 'o']

for cluster in range(6):
    cluster_data = df[df['Cluster_6b'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8

    )

# Adding labels and title
plt.title('Analysis B: k=6 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()


In [ ]:
########################################################################################
### K=5 Clusters B Analysis Plot
########################################################################################

df = pd.DataFrame(scatter_2023_5b)

cluster_0_size = scatter_2023_5b[scatter_2023_5b['Cluster_5b']==0].shape[0]
cluster_1_size = scatter_2023_5b[scatter_2023_5b['Cluster_5b']==1].shape[0]
cluster_2_size = scatter_2023_5b[scatter_2023_5b['Cluster_5b']==2].shape[0]
cluster_3_size = scatter_2023_5b[scatter_2023_5b['Cluster_5b']==3].shape[0]
cluster_4_size = scatter_2023_5b[scatter_2023_5b['Cluster_5b']==4].shape[0]


# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale


labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})', 
            f'Cluster 3 ({cluster_3_size})',
            f'Cluster 4 ({cluster_4_size})'
          ]
colors = ['green', 'gray', 'green', 'red', 'red']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
markers = ['P', 'P', 'P', 'o', 'o']

for cluster in range(5):
    cluster_data = df[df['Cluster_5b'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8
    )

# Adding labels and title
plt.title('Analysis B: k=5 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()

In [ ]:
########################################################################################
### K=4 Clusters B Analysis Plot
########################################################################################

df = pd.DataFrame(scatter_2023_4b)

# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale


cluster_0_size = scatter_2023_4b[scatter_2023_4b['Cluster_4b']==0].shape[0]
cluster_1_size = scatter_2023_4b[scatter_2023_4b['Cluster_4b']==1].shape[0]
cluster_2_size = scatter_2023_4b[scatter_2023_4b['Cluster_4b']==2].shape[0]
cluster_3_size = scatter_2023_4b[scatter_2023_4b['Cluster_4b']==3].shape[0]




labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})', 
            f'Cluster 3 ({cluster_3_size})',
          ]
colors = ['green', 'green', 'red', 'red']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
markers = ['P', 'P', 'o', 'o']

for cluster in range(4):
    cluster_data = df[df['Cluster_4b'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8

    )

# Adding labels and title
plt.title('Analysis B: k=4 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()

In [ ]:

########################################################################################
### K=3 Clusters B Analysis Plot
########################################################################################

df = pd.DataFrame(scatter_2023_3b)

# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale


cluster_0_size = scatter_2023_3b[scatter_2023_3b['Cluster_3b']==0].shape[0]
cluster_1_size = scatter_2023_3b[scatter_2023_3b['Cluster_3b']==1].shape[0]
cluster_2_size = scatter_2023_3b[scatter_2023_3b['Cluster_3b']==2].shape[0]



labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})'
        ]
colors = ['green', 'gray', 'red']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
markers = ['P', 'P', 'o']

for cluster in range(3):
    cluster_data = df[df['Cluster_3b'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8

    )

# Adding labels and title
plt.title('Analysis B: k=3 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()

In [ ]:
print('List of Schools in k-means k=3 and k=4 identified at risk of closure')
for i in data.query('(Cluster_3b == 2) and (Cluster_4b in (2,3))')['School'].unique().tolist(): 
    print(' - '+i)


In [ ]:
print('List of Schools in k-means k=3 and k=4 identified safe from closure')
for i in data.query('(Cluster_3b == 0) and (Cluster_4b in (0,1))')['School'].unique().tolist(): 
    print(' - '+i)

In [ ]:
########################################################################################
### Clustering Analysis C: Applying Weights to Features, Same Columns as B Analysis
########################################################################################


columns_to_cluster = [
    'General Education (BUDGET)', 
    'Other Grants (BUDGET)', 
    'Special Education (BUDGET)',
    'Total Budget (BUDGET)', 
    'Federal Title I (BUDGET)', 
    'Bilingual Education (ENROLLMENT)',
    'Free and Reduced Lunch (ENROLLMENT)', 
    'Special Education (ENROLLMENT)',
    'Total AAFTE* Enrollment (ENROLLMENT)', 
    'Budget Efficiency', 
    'Distance to Closest School (miles)'
]

weights = {
    'General Education (BUDGET)':               1, 
    'Other Grants (BUDGET)':                    1, 
    'Special Education (BUDGET)':               1,
    'Total Budget (BUDGET)':                    1, 
    'Federal Title I (BUDGET)':                 1, 
    'Bilingual Education (ENROLLMENT)':         1,
    'Free and Reduced Lunch (ENROLLMENT)':      1, 
    'Special Education (ENROLLMENT)':           1,
    'Total AAFTE* Enrollment (ENROLLMENT)':     1, 
    'Budget Efficiency' :                       2, 
    'Distance to Closest School (miles)':       4

}

imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data[columns_to_cluster])

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

i=0
for column, weight in weights.items():
    data_scaled[:,i] *= weight
    i+=1

sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    sse.append(kmeans.inertia_)


plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.title('Analysis C: Elbow Method to Determine Optimal k for Clustering')
plt.xlabel('Number of Clusters')
plt.ylabel('Sum of Squared Distances')
plt.show()


# Handle missing values by filling with the mean of each column
data_imputed = imputer.fit_transform(data[columns_to_cluster])

# Normalize the data

data_scaled = scaler.fit_transform(data_imputed)
df_scaled = pd.DataFrame(data_scaled, columns=columns_to_cluster)

for column, weight in weights.items():
    df_scaled[column] *= weight

# Apply K-means clustering with k=3
kmeans_3 = KMeans(n_clusters=3, random_state=42)
clusters_3 = kmeans_3.fit_predict(df_scaled)

# Apply K-means clustering with k=4
kmeans_4 = KMeans(n_clusters=4, random_state=42)
clusters_4 = kmeans_4.fit_predict(df_scaled)

# Apply K-means clustering with k=5
kmeans_5 = KMeans(n_clusters=5, random_state=42)
clusters_5 = kmeans_5.fit_predict(df_scaled)
                                  
# Apply K-means clustering with k=6
kmeans_6 = KMeans(n_clusters=6, random_state=42)
clusters_6 = kmeans_6.fit_predict(df_scaled)


# Add the cluster labels to the original dataframe
data['Cluster_3c'] = clusters_3
data['Cluster_4c'] = clusters_4
data['Cluster_5c'] = clusters_5
data['Cluster_6c'] = clusters_6

# Analyze the clusters by looking at the mean values of the features for each cluster
cluster_3c_summary = data.groupby('Cluster_3c')[columns_to_cluster].mean()
cluster_4c_summary = data.groupby('Cluster_4c')[columns_to_cluster].mean()
cluster_5c_summary = data.groupby('Cluster_5c')[columns_to_cluster].mean()
cluster_6c_summary = data.groupby('Cluster_6c')[columns_to_cluster].mean()

#cluster_3c_summary, cluster_4c_summary, cluster_5c_summary, cluster_6c_summary


In [ ]:
########################################################################################
## Observations from Clustering Analysis c using weighting: (1,1,1,1,1,1,1,1,1,2,4)
########################################################################################


#3 Clusters:
#Cluster 0: Moderate budgets across all categories, moderate to high enrollment numbers, and below-average budget efficiency, with schools located relatively close to each other.
#Cluster 1: High General Education budgets, average enrollment figures, and moderate budget efficiency, with schools located at greater distances.
#Cluster 2: Highest General Education budgets, lower enrollments, and higher budget efficiency, with schools moderately spaced apart.
# # # Cluster 0 is likely to be target for closure.
# # # Cluster 1 is likely not to be closed. 

#4 Clusters:
#Cluster 0: Lower overall budgets, lower enrollments, and moderate budget efficiency, with schools located very close to each other.
#Cluster 1: High General Education budgets, higher enrollments, and moderate budget efficiency, with schools located at the greatest distances.
#Cluster 2: High General Education budgets, average enrollments, and moderate budget efficiency, with schools moderately spaced apart.
#Cluster 3: Highest Total budgets, high enrollments, and the lowest budget efficiency, with schools located relatively close to each other.
# # # Cluster 0 most likely to be target for closure.
# # # Clusters 1 and 2 likely not to be closed.

#5 Clusters:
#Cluster 0: Lower overall budgets, lower enrollments, and moderate budget efficiency, with schools located very close to each other.
#Cluster 1: High General Education budgets, high enrollments, and moderate budget efficiency, with schools located at the greatest distances.
#Cluster 2: Highest General Education budgets, the highest enrollments, and higher budget efficiency, with schools moderately spaced apart
#Cluster 3: High budgets, higher enrollments, and the lowest budget efficiency, with schools located relatively close to each other
#Cluster 4: Lowest General Education budgets, moderate enrollments, and moderate budget efficiency, with schools moderately spaced apart.
# # # Clusters 0 and 3 most likely to be targets for closure.
# # # Clusters 1 and 2 likely not to be closed. 

#6 Clusters:
#Cluster 0: Lower overall budgets, lower enrollments, and moderate budget efficiency, with schools located very close to each other.
#Cluster 1: Highest Total budgets, highest enrollments, and low budget efficiency, with schools located at the greatest distances
#Cluster 2: High General Education budgets, high enrollments, and the highest budget efficiency, with schools moderately spaced apart.
#Cluster 3: Have high budgets, high enrollments, and the lowest budget efficiency, with schools located relatively close to each other.
#Cluster 4: Moderate budgets, moderate enrollments, and low budget efficiency, with schools moderately spaced apart.
#Cluster 5: Moderate budgets, moderate enrollments, and moderate budget efficiency, with schools located at relatively greater distances.
# # # Clusters 0 and 3 most likely to be targets for closure.
# # # Clusters 1 and 2 most likely not to be closed.

In [ ]:
scatter_6 = data[['School','Year','Cluster_6c', 'Distance to Closest School (miles)', 'Budget Efficiency']]
scatter_5 = data[['School','Year','Cluster_5c', 'Distance to Closest School (miles)', 'Budget Efficiency']]
scatter_4 = data[['School','Year','Cluster_4c', 'Distance to Closest School (miles)', 'Budget Efficiency']]
scatter_3 = data[['School','Year','Cluster_3c', 'Distance to Closest School (miles)', 'Budget Efficiency']]

scatter_2023_6c = scatter_6[scatter_6['Year']=='2023-24'].sort_values(by='Cluster_6c')
scatter_2022_6c = scatter_6[scatter_6['Year']=='2022-23'].sort_values(by='Cluster_6c')
scatter_2021_6c = scatter_6[scatter_6['Year']=='2021-22'].sort_values(by='Cluster_6c')

scatter_2023_5c = scatter_5[scatter_5['Year']=='2023-24'].sort_values(by='Cluster_5c')
scatter_2022_5c = scatter_5[scatter_5['Year']=='2022-23'].sort_values(by='Cluster_5c')
scatter_2021_5c = scatter_5[scatter_5['Year']=='2021-22'].sort_values(by='Cluster_5c')

scatter_2023_4c = scatter_4[scatter_4['Year']=='2023-24'].sort_values(by='Cluster_4c')
scatter_2022_4c = scatter_4[scatter_4['Year']=='2022-23'].sort_values(by='Cluster_4c')
scatter_2021_4c = scatter_4[scatter_4['Year']=='2021-22'].sort_values(by='Cluster_4c')

scatter_2023_3c = scatter_3[scatter_3['Year']=='2023-24'].sort_values(by='Cluster_3c')
scatter_2022_3c = scatter_3[scatter_3['Year']=='2022-23'].sort_values(by='Cluster_3c')
scatter_2021_3c = scatter_3[scatter_3['Year']=='2021-22'].sort_values(by='Cluster_3c')

In [ ]:
########################################################################################
### K=6 Clusters C Analysis Plot
########################################################################################


df = pd.DataFrame(scatter_2023_6c)

# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale

cluster_0_size = scatter_2023_6c[scatter_2023_6c['Cluster_6c']==0].shape[0]
cluster_1_size = scatter_2023_6c[scatter_2023_6c['Cluster_6c']==1].shape[0]
cluster_2_size = scatter_2023_6c[scatter_2023_6c['Cluster_6c']==2].shape[0]
cluster_3_size = scatter_2023_6c[scatter_2023_6c['Cluster_6c']==3].shape[0]
cluster_4_size = scatter_2023_6c[scatter_2023_6c['Cluster_6c']==4].shape[0]
cluster_5_size = scatter_2023_6c[scatter_2023_6c['Cluster_6c']==5].shape[0]


labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})', 
          f'Cluster 3 ({cluster_3_size})',
          f'Cluster 4 ({cluster_4_size})',
          f'Cluster 5 ({cluster_5_size})'
          ]
colors = ['red', 'green', 'green', 'red', 'gray', 'gray']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
#markers = ['o', 's', '^', 'P', 'D', 'X']
markers = ['o', 'P', 'P', 'o', 'P', 'P']

for cluster in range(6):
    cluster_data = df[df['Cluster_6c'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8

    )

# Adding labels and title
plt.title('Analysis C: k=6 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()


In [ ]:
########################################################################################
### K=5 Clusters C Analysis Plot
########################################################################################


df = pd.DataFrame(scatter_2023_5c)

# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale

cluster_0_size = scatter_2023_5c[scatter_2023_5c['Cluster_5c']==0].shape[0]
cluster_1_size = scatter_2023_5c[scatter_2023_5c['Cluster_5c']==1].shape[0]
cluster_2_size = scatter_2023_5c[scatter_2023_5c['Cluster_5c']==2].shape[0]
cluster_3_size = scatter_2023_5c[scatter_2023_5c['Cluster_5c']==3].shape[0]
cluster_4_size = scatter_2023_5c[scatter_2023_5c['Cluster_5c']==4].shape[0]


labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})', 
            f'Cluster 3 ({cluster_3_size})',
            f'Cluster 4 ({cluster_4_size})'
          ]
colors = ['red', 'green', 'green', 'red', 'gray']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
#markers = ['o', 's', '^', 'P', 'D', 'X']
markers = ['o', 'P', 'P', 'o', 'P']

for cluster in range(5):
    cluster_data = df[df['Cluster_5c'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8

    )

# Adding labels and title
plt.title('Analysis C: k=5 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()

In [ ]:
########################################################################################
### K=4 Clusters C Analysis Plot
########################################################################################


df = pd.DataFrame(scatter_2023_4c)

# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale

cluster_0_size = scatter_2023_4c[scatter_2023_4c['Cluster_4c']==0].shape[0]
cluster_1_size = scatter_2023_4c[scatter_2023_4c['Cluster_4c']==1].shape[0]
cluster_2_size = scatter_2023_4c[scatter_2023_4c['Cluster_4c']==2].shape[0]
cluster_3_size = scatter_2023_4c[scatter_2023_4c['Cluster_4c']==3].shape[0]
cluster_4_size = scatter_2023_4c[scatter_2023_4c['Cluster_4c']==4].shape[0]


labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})', 
            f'Cluster 3 ({cluster_3_size})'
          ]
colors = ['red', 'green', 'green', 'gray']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
#markers = ['o', 's', '^', 'P', 'D', 'X']
markers = ['o', 'P', 'P', 'P']

for cluster in range(4):
    cluster_data = df[df['Cluster_4c'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8

    )

# Adding labels and title
plt.title('Analysis C: k=4 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()

In [ ]:
########################################################################################
### K=3 Clusters C Analysis Plot
########################################################################################


df = pd.DataFrame(scatter_2023_3c)

# Function to add jitter
def add_jitter(arr, scale=0.01):
    return arr + np.random.randn(len(arr)) * scale

cluster_0_size = scatter_2023_3c[scatter_2023_3c['Cluster_3c']==0].shape[0]
cluster_1_size = scatter_2023_3c[scatter_2023_3c['Cluster_3c']==1].shape[0]
cluster_2_size = scatter_2023_3c[scatter_2023_3c['Cluster_3c']==2].shape[0]


labels = [
          f'Cluster 0 ({cluster_0_size})', 
          f'Cluster 1 ({cluster_1_size})', 
          f'Cluster 2 ({cluster_2_size})'
          ]
colors = ['red', 'gray', 'green']


# Plotting the k=6 clusters with enhancements
plt.figure(figsize=(12, 8))

# Marker styles
#markers = ['o', 's', '^', 'P', 'D', 'X']
markers = ['o', 'P', 'P']

for cluster in range(3):
    cluster_data = df[df['Cluster_3c'] == cluster]
    plt.scatter(
        cluster_data['Budget Efficiency'], #add_jitter(cluster_data['Budget Efficiency'], 100), 
        cluster_data['Distance to Closest School (miles)'], #add_jitter(cluster_data['Distance to Closest School (miles)'], 0.01),
        color=colors[cluster],
        marker=markers[cluster],
        label=labels[cluster],
        alpha=0.8

    )

# Adding labels and title
plt.title('Analysis C: k=3 Clusters: Budget Efficiency vs. Distance to Closest School')
plt.xlabel('Budget Efficiency')
plt.ylabel('Distance to Closest School (miles)')

# Creating a legend
plt.legend()

plt.show()

In [ ]:
print('Analysis C: List of Schools in k-means k=3 and k=4 identified at risk of closure')
for i in data.query('(Cluster_3c == 0) and (Cluster_4c == 0)')['School'].unique().tolist(): 
    print(' - '+i)

In [ ]:
print('Analysis C: List of Schools in k-means k=3 and k=4 identified safe from closure')
for i in data.query('(Cluster_3c == 1) and (Cluster_4c in (1,2))')['School'].unique().tolist(): 
    print(' - '+i)

In [ ]:
facilities = get_facility_table()

In [ ]:

d1 = pd.merge(data,facilities,on='School',how='left')

d1['Disadvantage Score'] = (d1['Free and Reduced Lunch (ENROLLMENT)'] + d1['Bilingual Education (ENROLLMENT)'] + d1['Special Education (ENROLLMENT)'])/d1['Total AAFTE* Enrollment (ENROLLMENT)']
d1.fillna(0, inplace=True)

d1['Year'] = d1['Year'].str.split('-').str[0]
d1_orig = d1.copy()
d1.drop(columns=['Cluster_3a','Cluster_3b', 'Cluster_4a','Cluster_4b', 'Cluster_5a', 'Cluster_5b', 'Cluster_6a', 'Cluster_6b', 'Cluster_3c', 'Cluster_4c', 'Cluster_5c', 'Cluster_6c'], inplace=True)
d1.drop(columns=['Building Area (Gross sf)','Building Area (SCAP Recognized sf)','Site Area (acre)','Date of Construction','School'], inplace=True)
d1.drop(columns=['Full Address','Closest School', 'Distance to Closest School (miles)', 'Address','Landmark','Date of Last Full Reno','Levy (1985-2019)','Classification','Latitude','Longitude','Use','Budget Efficiency'], inplace=True)
columns_to_drop = [col for col in d1.columns if col.endswith('(BUDGET)') and col not in ['Total Budget (BUDGET)']]
d1.drop(columns=columns_to_drop, inplace=True)

d2 = d1.query('Year in ("2023")').drop(columns=['Year'])
d1 = d1.query('Year in ("2022","2021")').drop(columns=['Year'])



X = d1.copy()
X.drop(columns=['Total Budget (BUDGET)'], inplace=True)

y = d1.copy()
y = y['Total Budget (BUDGET)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

#mse = mean_squared_error(y_test, y_pred)
#rmse = root_mean_squared_error(y_test, y_pred)
#r2 = r2_score(y_test, y_pred)

#print(f"Mean Squared Error: {mse}")
#print(f"Root Mean Squared Error: {mse}")
#print(f"R^2 Score: {r2}")

for column in X_train.columns:
    X_train[column] = pd.to_numeric(X_train[column], errors='coerce')
X_train = X_train.dropna()

X_train_sm = sm.add_constant(X_train)  # Adds a constant term to the predictor
ols_model = sm.OLS(y_train, X_train_sm).fit()
print(ols_model.summary())




In [ ]:
# Y-Intercept
intercept = ols_model.params[0]
formatted_intercept = f"${intercept:,.2f}"
print(f"Y-Intercept, Representing the Baseline Cost for Operating an Elementary School : {formatted_intercept}")


# Residual plot
plt.figure(figsize=(10, 6))
plt.scatter(ols_model.fittedvalues, ols_model.resid)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# QQ plot
sm.qqplot(ols_model.resid, line='s')
plt.title('QQ Plot')
plt.show()


# Partial regression plots
fig = plt.figure(figsize=(12, 8))
sm.graphics.plot_partregress_grid(ols_model, fig=fig)
plt.show()

In [ ]:
for column in d2.columns:
    d2[column] = pd.to_numeric(d2[column], errors='coerce')

new_data = d2.dropna().reset_index(drop=True)
new_data_sm = sm.add_constant(new_data.drop(columns=['Total Budget (BUDGET)'])) 
new_predictions = pd.DataFrame(ols_model.predict(new_data_sm),columns=['Predicted Total Budget (BUDGET)'])
new_predictions['Predicted Total Budget (BUDGET)'] = new_predictions['Predicted Total Budget (BUDGET)'].apply(lambda x: f"{x:.2f}").astype(float)
new_data = new_data.join(new_predictions)
new_data['Excess Total Budget'] = (new_data['Total Budget (BUDGET)'] - new_data['Predicted Total Budget (BUDGET)']).apply(lambda x: f"{x:.2f}").astype(float)
new_data['Excess Budget per Student'] = (new_data['Excess Total Budget'] / new_data['Total AAFTE* Enrollment (ENROLLMENT)']).apply(lambda x: f"{x:.2f}").astype(float)


In [ ]:
performance_2024 = d1_orig.query('Year in ("2023")').reset_index(drop=True).join(new_data,rsuffix='_y')

In [ ]:

columns_to_drop = [col for col in performance_2024.columns if col.endswith('_y')]
performance_2024.drop(columns=columns_to_drop, inplace=True)
performance_2024['Building Area (Gross sf)'] = performance_2024['Building Area (Gross sf)'].apply(convert_to_float)
performance_2024['Building Area per Student (sf)'] = (performance_2024['Building Area (Gross sf)'] / performance_2024['Total AAFTE* Enrollment (ENROLLMENT)']).apply(lambda x: f"{x:.2f}").astype(float)
capacity_data = extract_tables_from_pdf(facilities_master_plan,43)[0][2:].rename(columns={'Total\nEstimated\nOperational\nCapacity': 'Capacity'})[['School Name','Capacity']]
capacity_data['School'] = capacity_data['School Name'].apply(rename_school)
capacity_data = capacity_data.fillna('None')[['School','Capacity']]

performance_2024 = pd.merge(performance_2024,capacity_data, on='School', how='left')

performance_2024['Excess Capacity'] = performance_2024['Capacity'].astype(int) - performance_2024['Total AAFTE* Enrollment (ENROLLMENT)'].astype(int)
performance_2024['Capacity Percent'] = (performance_2024['Total AAFTE* Enrollment (ENROLLMENT)'] / performance_2024['Capacity'].astype(int)).apply(lambda x: f"{x:.2f}").astype(float)



In [ ]:
performance_2024.query('School == "John Stanford International Elementary"').transpose()


In [ ]:
performance_2024.to_csv("data/performance_data_2023.csv")

In [ ]:
school_matrix = pd.DataFrame(0,index=facilities['School'].to_list(),columns=facilities['School'].to_list())

In [ ]:
y1 = extract_tables_from_pdf(attendance_area_report, 2)[0]
y1.columns[0].split('\n')


In [ ]:

y1.iloc[1,0].split('\n')
tz = create_df_from_split_string(y1.iloc[1,0].split('\n'))
tz['School_new'] = tz['School'].apply(rename_school)
tz.rename(columns={'School_new':'School', 'School':'Old School'}, inplace=True)
tz.drop(columns=['Old School'], inplace=True)
tz
#tz = tz.fillna('None').query('School != "None"').reset_index(drop=True)


In [ ]:

def redistribute_students(student_counts, redistribution_matrix, closed_schools):
    # Convert input lists to numpy arrays for easy manipulation
    student_counts = np.array(student_counts, dtype=float)
    redistribution_matrix = np.array(redistribution_matrix, dtype=float)
    closed_schools = set(closed_schools)

    # Initialize current student counts
    current_student_counts = np.copy(student_counts)
    num_schools = len(student_counts)
    
    while True:
        new_counts = np.copy(current_student_counts)
        any_redistributed = False
        
        for school in closed_schools:
            if current_student_counts[school] > 0:
                any_redistributed = True
                total_redistributed = current_student_counts[school]
                
                # Adjust redistribution percentages to only consider open schools
                redistribution_percentages = np.copy(redistribution_matrix[school])
                redistribution_percentages[list(closed_schools)] = 0
                total_percentage = np.sum(redistribution_percentages)
                
                if total_percentage > 0:
                    redistribution_percentages /= total_percentage
                
                for i in range(num_schools):
                    if i not in closed_schools:
                        new_counts[i] += total_redistributed * redistribution_percentages[i]
                
                new_counts[school] = 0
        
        if not any_redistributed:
            break
        
        current_student_counts = new_counts
    
    return current_student_counts

# Example usage
student_counts = [100, 200, 150, 50]
redistribution_matrix = [
    [0.0, 0.3, 0.5, 0.2],
    [0.4, 0.0, 0.4, 0.2],
    [0.3, 0.2, 0.0, 0.5],
    [0.5, 0.3, 0.2, 0.0]
]
closed_schools = [1, 2]

new_student_counts = redistribute_students(student_counts, redistribution_matrix, closed_schools)
print(f"New student counts: {new_student_counts}")
print(f"Total students: {np.sum(new_student_counts)}")


In [ ]:
import pandas as pd

def update_first_dataframe(df1, df2, column_name):
    """
    Updates the 'df1' dataframe with values from 'df2' based on a specified column.
    
    Parameters:
    df1 (pd.DataFrame): The dataframe to be updated.
    df2 (pd.DataFrame): The dataframe providing the updated values.
    column_name (str): The column name to base the updates on.
    
    Returns:
    pd.DataFrame: The updated 'df1' dataframe.
    """
    # Ensure the column exists in both dataframes
    if column_name not in df1.columns or column_name not in df2.columns:
        raise ValueError(f"Column '{column_name}' not found in both dataframes.")
    
    # Iterate over the rows of the second dataframe
    for idx, row in df2.iterrows():
        if idx in df1.index:
            df1.at[idx, column_name] = row[column_name]
        else:
            df1.loc[idx] = row
    
    return df1

# Example usage:
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [1, 2, 3], 'B': [7, 8, 9]})
column_name = 'B'
updated_df1 = update_first_dataframe(df1, df2, column_name)
print(updated_df1)


,School,Weight
0,Adams Elementary,0.000000
1,Alki Elementary,0.000000
2,Arbor Heights Elementary,0.000000
3,Daniel Bagley Elementary,0.006494
4,Beacon Hill International Elementary,0.000000
...,...,...
57,View Ridge Elementary,0.004870
58,Viewlands Elementary,0.006494
59,Wedgwood Elementary,0.019481
60,West Seattle Elementary,0.000000


NameError: name 'schools' is not defined

In [170]:
redistribute_students_vector('John Stanford International Elementary')

,School,Weight
0,Adams Elementary,0.028384
1,Alki Elementary,0.000000
2,Arbor Heights Elementary,0.000000
3,Daniel Bagley Elementary,0.034934
4,Beacon Hill International Elementary,0.000000
...,...,...
57,View Ridge Elementary,0.021834
58,Viewlands Elementary,0.010917
59,Wedgwood Elementary,0.017467
60,West Seattle Elementary,0.000000
